<a href="https://colab.research.google.com/github/Alilson2/Projeto_IA/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr # Ler arquivos netcdf
from google.colab import drive
import seaborn as sns
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Abrindo arquivo netcdf
dados = xr.open_dataset('/content/drive/MyDrive/ERA5Land_exemplo.nc')
dados

<xarray.Dataset> Size: 51MB
Dimensions:    (time: 12, latitude: 501, longitude: 531)
Coordinates:
  * time       (time) datetime64[ns] 96B 2000-01-01 2000-02-01 ... 2000-12-01
  * latitude   (latitude) float32 2kB 12.0 11.9 11.8 11.7 ... -37.8 -37.9 -38.0
  * longitude  (longitude) float32 2kB -85.0 -84.9 -84.8 ... -32.2 -32.1 -32.0
Data variables:
    d2m        (time, latitude, longitude) float64 26MB ...
    t2m        (time, latitude, longitude) float64 26MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-25 21:58:15 GMT by grib_to_netcdf-2.24.0: /opt/ecmw...

In [ ]:
dados.info()

xarray.Dataset {
dimensions:
	time = 12 ;
	latitude = 501 ;
	longitude = 531 ;

variables:
	float64 d2m(time, latitude, longitude) ;
		d2m:units = K ;
		d2m:long_name = 2 metre dewpoint temperature ;
	float64 t2m(time, latitude, longitude) ;
		t2m:units = K ;
		t2m:long_name = 2 metre temperature ;
	float32 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:long_name = longitude ;
	float32 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:long_name = latitude ;
	datetime64[ns] time(time) ;
		time:long_name = time ;

// global attributes:
	:Conventions = CF-1.6 ;
	:history = 2024-07-25 21:58:15 GMT by grib_to_netcdf-2.24.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -S param -o /cache/tmp/7b9350be-5724-4c3b-8659-c3c14051b994-adaptor.mars.internal-1721944694.7020206-32523-11-tmp.nc /cache/tmp/7b9350be-5724-4c3b-8659-c3c14051b994-adaptor.mars.internal-1721944669.2716146-32523-10-tmp.grib ;
}

In [ ]:
# --- 2. Verificar se há a coordenada temporal ---
if "time" not in dados.coords:
    raise ValueError("O dataset não contém uma coordenada temporal chamada 'time'.")

# --- 3. Converter o eixo temporal para pandas.DatetimeIndex ---
tempo = pd.to_datetime(dados["time"].values)

# --- 4. Criar DataFrame com componentes temporais ---
df_tempo = pd.DataFrame({
    "timestamp": tempo,
    "timestamp_segundos": tempo.view("int64") / 1e9,   # converte para segundos desde 1970
    "ano": tempo.year,
    "mes": tempo.month,
    "dia": tempo.day,
    "hora": tempo.hour,
    "minuto": tempo.minute,
    "segundo": tempo.second,
    "dia_semana": tempo.dayofweek,
    "dia_do_ano": tempo.dayofyear
})

print(df_tempo)

    timestamp  timestamp_segundos   ano  mes  dia  hora  minuto  segundo  \
0  2000-01-01         946684800.0  2000    1    1     0       0        0   
1  2000-02-01         949363200.0  2000    2    1     0       0        0   
2  2000-03-01         951868800.0  2000    3    1     0       0        0   
3  2000-04-01         954547200.0  2000    4    1     0       0        0   
4  2000-05-01         957139200.0  2000    5    1     0       0        0   
5  2000-06-01         959817600.0  2000    6    1     0       0        0   
6  2000-07-01         962409600.0  2000    7    1     0       0        0   
7  2000-08-01         965088000.0  2000    8    1     0       0        0   
8  2000-09-01         967766400.0  2000    9    1     0       0        0   
9  2000-10-01         970358400.0  2000   10    1     0       0        0   
10 2000-11-01         973036800.0  2000   11    1     0       0        0   
11 2000-12-01         975628800.0  2000   12    1     0       0        0   

    dia_sem